# Residual Network

Для классификации предобработанных звуковых файлов используем популярную сегодня глубокую нейронную сеть под названием Residual Network (сокращённая сеть), смысл которой в том, что через каждые несколько слоёв в сумматоры нейронов выходного слоя блока сети подаётся входной вектор. Это позволяет сократить количество слоёв при сравнимой глубине нелинейности получаемого функционала.

Residual Networks, представлены в статье [He et al.](https://arxiv.org/pdf/1512.03385.pdf).

In [3]:
import pandas as pd
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
#import pydot
import tensorflow as tf
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from PIL import Image

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


## 1 - The problem of very deep neural networks

Главное преимущество очень глубоких сетей - то, что они способны вычислять очень сложный функционал. Это позволяет также выделять признаки разных уровней абстракций - от краёв объектов на изображениях (на первых слоях) до очень сложных признаков (на более глубоких слоях). Но использование более глубокой сети не всегда выручает. Препятствием является то обстоятельство, что градиент при приближении к первым слоям стремится к нулю, и такая сеть обучается очень медленно. 

Амплитуда градиента на разных слоях с ростом числа итераций показана на рисунке.

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> **Рисунок 1** </u><font color='purple'>  : **"Размывание градиента"** <br> Скорость обучения падает значительно для первых слоёв с ростом числа итераций </center></caption>

Эта проблема может быть решена с Residual Network.

## 2 - Построение ResNet

В ResNets, "сокращение" или "пустое соединение" позволяет градиентному спуску сходиться намного быстрее.

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> **Рисунок 2** </u><font color='purple'>  : Блок ResNet, демонстрирующий **пустое соединение** <br> </center></caption>

Картинка слева показывает "основной маршрут" распространения сигнала по сети. На картинке справа добавляется сокращение основного маршрута. Объединяя последовательно такие блоки, можно построить очень глубокую сеть. 

В ResNet используются два типа блоков, в зависимости от того, различаются ли входные и выходные размерности этих блоков. Мы разработаем оба типа блоков.

### 2.1 - Основной блок

Основной блок - это стандартный блок, используемый в ResNet в случае, когда входные и выходные размерности совпадают. Чтобы показать, что происходит внутри такого блока, нарисуем следующую диаграмму, отображающую основные части этого блока.

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 3** </u><font color='purple'>  : **Основной блок** Пустое поединение пропускается через 2 слоя. </center></caption>

Верхний путь - и есть "сокращение". Нижний путь - это "основной путь" сигнала. Как показано на диаграмме, мы делаем внутри этого маршрута шаги CONV2D (2D свёртка) и ReLU (активация ReLU) в каждом слое. Для ускорения сходимости мы добавляем нормализацию (шаг BatchNorm). Не нужно волноваться о сложности реализации - BatchNorm реализуется всего одной строкой в Keras! 

В этом примере мы сделаем сокращение не через 2 слоя, а через 3. Это быдет выглядеть вот так: 

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 4** </u><font color='purple'>  : **Основной блок.** Пустое соединение через 3 слоя.</center></caption>

Распишем шаги по отдельности.

Первая компонента основного пути: 
- Первый слой CONV2D имеет $F_1$ фильтров размерности (1,1) со сдвигом (1,1). Отступы устанавливаем на "valid" и присваиваем имя `conv_name_base + '2a'`. 
- Первый слой BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '2a'`.
- Затем применяем ReLU-функцию для активации. Не будем задавать ей имя и гиперпараметры. 

Вторая компонента основного пути: 
- Второй слой CONV2D имеет $F_2$ фильтров размерности (f,f) со сдвигом (1,1). Отступы устанавливаем на "same" и присваиваем имя `conv_name_base + '2b'`.
- Второй слой BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '2b'`.
- Затем применяем ReLU-функцию для активации. Не будем задавать ей имя и гиперпараметры.

Третья компонента основного пути: 
- Третий слой CONV2D имеет $F_3$ фильтров размерности (1,1) со сдвигом (1,1). Отступы устанавливаем на "valid" и присваиваем имя `conv_name_base + '2c'`. 
- Третий слой BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '2с'`.
После этого слоя нет ReLU-активации.

Последний шаг: 
- Сливаем выход со входом по короткому пути.
- Применяем ReLU-функцию активации. Не будем задавать ей имя и гиперпараметры. 

In [4]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Реазизация основного блока с Рисунка 3
    
    Аргументы:
    X -- Входной тензор размерности (m, n_H_prev, n_W_prev, n_C_prev)
    f -- целое число, определяющее размер среднего свёрточного (CONV) окна в основном пути
    filters -- список целых чисел, определяющий количество фильтров в CONV-слоях основного пути
    stage -- целое число, используемое для нумерации слоёв, зависит от позиции слоя в сети
    block -- строка/символ, используемое для наименования слоёв, зависит от позиции слоя в сети
    
    Выход:
    X -- выход основного блока, тензор размерности (n_H, n_W, n_C)
    """
    
    # Определяем базис имён
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Извлекаем фильтры
    F1, F2, F3 = filters
    
    # Сохраняем входной тензор. YОн понадобится позже для добавления его к основному пути. 
    X_shortcut = X
    
    # Первая компонента основного пути
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Вторая компонента основного пути
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Третья компонента основного пути
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Последний шаг: добавляем входной тензор к основному пути и пропускаем через RELU-активацию
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

## 2.2 - Свёрточный блок

Мы реализовали основной блок. Далее идёт "свёрточный блок" - это другой тип блока в ResNet. Мы будем его использовать, когда входные и выходные размерности не будут совпадать. Различие здесь в том, что в коротком пути присутствует свёрточный (CONV2D) слой: 

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 4** </u><font color='purple'>  : **Свёрточный блок** </center></caption>

Слой CONV2D в коротком пути используется для измерения размера входного тензора $x$. Этот слой не использует нелинейную активационную функцию. Его основная функция - уменьшение размерности.

Детально этот блок выглядит так. 

Первая компонента основного пути: 
- Первый слой CONV2D имеет $F_1$ фильтров размерности (1,1) со сдвигом (s,s). Отступы устанавливаем на "valid" и присваиваем имя `conv_name_base + '2a'`. 
- Первый слой BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '2a'`.
- Затем применяем ReLU-функцию для активации. Не будем задавать ей имя и гиперпараметры. 

Вторая компонента основного пути: 
- Второй слой CONV2D имеет $F_2$ фильтров размерности (f,f) со сдвигом (1,1). Отступы устанавливаем на "same" и присваиваем имя `conv_name_base + '2b'`.
- Второй слой BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '2b'`.
- Затем применяем ReLU-функцию для активации. Не будем задавать ей имя и гиперпараметры.

Третья компонента основного пути: 
- Третий слой CONV2D имеет $F_3$ фильтров размерности (1,1) со сдвигом (1,1). Отступы устанавливаем на "valid" и присваиваем имя `conv_name_base + '2c'`. 
- Третий слой BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '2с'`.
После этого слоя нет ReLU-активации.

Короткий путь:
- Слой CONV2D имеет $F_3$ фильтров размерности (1,1) со сдвигом (s,s). Отступы устанавливаем на "valid" и присваиваем имя `conv_name_base + '1'`.
- BatchNorm нормализует значения выходов. Его надо назвать `bn_name_base + '1'`.

Последний шаг: 
- Сливаем выход со входом по короткому пути.
- Применяем ReLU-функцию активации. Не будем задавать ей имя и гиперпараметры. 

In [5]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Реализация свёрточного блока с Рисунка 4
    
    Аргументы:
    X -- входной тензор размерности (m, n_H_prev, n_W_prev, n_C_prev)
    f -- целое число, определяющее размер среднего свёрточного (CONV) окна в основном пути
    filters -- список целых чисел, определяющий количество фильтров в CONV-слоях основного пути
    stage -- целое число, используемое для нумерации слоёв, зависит от позиции слоя в сети
    block -- строка/символ, используемое для наименования слоёв, зависит от позиции слоя в сети
    s -- целое число, указывающее размер сдвига
    
    Выход:
    X -- выход основного блока, тензор размерности (n_H, n_W, n_C)
    """
    
    # Определяем базис имён
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Извлекаем фильтры
    F1, F2, F3 = filters
    
    # Сохраняем входной тензор
    X_shortcut = X


    ##### Основной путь #####
    # Первая компонента основного пути
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Вторая компонента основного пути
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Третья компонента основного пути
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### Короткий путь ####
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Последний шаг: добавляем входной тензор к основному пути и пропускаем через RELU-активацию
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
        
    return X

## 3 - Строим модель ResNet (50 слоёв)

Мы реализовали необходимые блоки ResNet. Следующий рисунок отображает детально архитектуру нашей сети. "ID BLOCK" на диаграмме соответствует основному блоку, а "ID BLOCK x3" означает 3 последовательных основных блока.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> **Рисунок 5** </u><font color='purple'>  : **Модель ResNet-50** </center></caption>

В деталях модель ResNet-50 такова:
- Добавление нулевой рамки ко входам размером (3,3)
- Стадия 1:
    - 2D свёртка имеет 64 фильтра размером (7,7) со сдвигом (2,2). Называется "conv1".
    - BatchNorm применятся к входным значениям.
    - MaxPooling использует окно (3,3) и сдвиг (2,2).
- Стадия 2:
    - Свёрточный блок использует набор из трёх фильтров размерности [64,64,256], "f" - 3, "s" - 1 и метка блока - "a".
    - 2 основных блока используют набор трёх фильтров размерности [64,64,256], "f" - 3 и и метки блока "b" и "c".
- Стадия 3:
    - Свёрточный блок использует набор из трёх фильтров размерности [128,128,512], "f" - 3, "s" - 2 и метка блока - "a".
    - 3 основных блока используют набор трёх фильтров размерности [128,128,512], "f" - 3 и и метки блока "b", "c", "d".
- Стадия 4:
    - Свёрточный блок использует набор из трёх фильтров размерности [256,256,1024], "f" - 3, "s" - 2 и метка блока - "a".
    - 5 основных блоков используют набор трёх фильтров размерности [256,256,1024], "f" - 3 и и метки блока "b", "c", "d", "e", "f".
- Стадия 5:
    - Свёрточный блок использует набор из трёх фильтров размерности [512,512,2048], "f" - 3, "s" - 2 и метка блока - "a".
    - 2 основных блока используют набор трёх фильтров размерности [512,512,2048], "f" - 3 и и метки блока "b", "c".
- 2D Average Pooling использует окно размером (2,2) и имя "avg_pool".
- Слой, уменьшающий пространственную размерность, не имеет имени и гиперпараметров.
- Полносвязные (Dense) уменьшают размерность до количества классов и используют softmax-активацию. Его имя `'fc' + str(classes)`.

In [6]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (100, 100, 1), classes = 10):
    """
    Реализация модели ResNet50 следующей архитектуры:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Аргументы:
    input_shape -- Гистограммы (изображения) из набора данных
    classes -- целое число, количество классов

    Выход:
    model -- модель типа Model() в Keras
    """
    
    # Определяем входной вектор заданной размерности
    X_input = Input(input_shape)

    
    # Добавляем рамку из нулей
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Стадия 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Стадия 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Стадия 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Стадия 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Стадия 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), strides=None)(X)

    # Выходной слой
    X = Flatten()(X)
    X = Dense(classes, activation='linear', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Создаём модель
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

Запуск этого кода строит граф модели.

In [7]:
model = ResNet50(input_shape = (100, 100, 1), classes = 4)

In [9]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 1)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 106, 106, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 50, 64)   3200        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 50, 50, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

Компиляция модели

In [10]:
X_train = np.zeros([1000,100,100,1])

In [11]:
Y_train = pd.get_dummies(np.array(np.random.rand(1000)*4,dtype=np.int32))

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Теперь осталось загрузить данные и обучить сеть!

In [13]:
model.fit(X_train, Y_train, epochs = 1, batch_size = 32)

Epoch 1/1
1000/1000 [==============================] - 66s 66ms/step - loss: nan - accuracy: 0.2410


### Список литературы 

ResNet алгоритм описан в работе He et al. (2015). Реализация, использованная здесь, взята из открытого репозитория github Франсуа Шолле (Francois Chollet): 

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's github repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py


In [ ]:
ResNet-100
ResNet-150